# 1. Import libray

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import missingno as msno
import matplotlib.pyplot as plt
#I had the same issue.

#There seems to be an issue with the source from which Plotly is loaded. Changing the notebook mode might help.

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

from lightgbm import LGBMClassifier
#import lazypredict
#from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 2. Data

## 2.1 Reaning in dataset

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv") # read train data
train

In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv") # test data 
test

In [ ]:
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv") # test data 
submission

# 3. Quick view of data 

## 3.1 train data 

In [ ]:
train_info = train.info()

In [ ]:
def get_dataframe_info(df):
    """
    input
       df -> DataFrame
    output
       df_null_counts -> DataFrame Info (sorted)
    """

    df_types = pd.DataFrame(df.dtypes)
    df_nulls = df.count()
    
    df_null_count = pd.concat([df_types, df_nulls], axis=1)
    df_null_count = df_null_count.reset_index()
    
    # Reassign column names
    col_names = ["features", "types", "non_null_counts"]
    df_null_count.columns = col_names
    
    # Add this to sort
    df_null_count = df_null_count.sort_values(by=["types"], ascending=False)
    
    return df_null_count

In [ ]:
get_dataframe_info(train)

In [ ]:
train.columns #show colums 

In [ ]:
train.nunique() # get unquie data number

In [ ]:
train.isnull().any() # get if data has missing data or not

In [ ]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}') 
print(f'\033[94mNumber of columns in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber missing values in train data: {sum(train.isna().sum())}')

In [ ]:
print(f'\033[94m')
print(train.isna().sum().sort_values(ascending = False)) # get total numer of midssing data in each columns

In [ ]:
train.describe() # get statistics value in each columns

## 3.2 Quick view of train data 

In [ ]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values  in test data: {sum(test.isna().sum())}')

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False)))

In [ ]:
test.describe()

# 4. EDA

In [ ]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values  in test data: {sum(test.isna().sum())}')

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False)))

In [ ]:
test.describe()

## 4.2 Overview of Data

In [ ]:
train.drop(["PassengerId"] , axis = 1 , inplace = True)
test.drop(["PassengerId"] , axis = 1 , inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col != TARGET]
RANDOM_STATE = 12 

In [ ]:
train.iloc[:, :-1].describe()

In [ ]:
train.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')

## 4.3 Nan value Distribtion

### 4.3.1 Coulums wise Null value Distribution

In [ ]:
test_null = pd.DataFrame(test.isna().sum())
test_null = test_null.sort_values(by = 0, ascending=False)
test_null

In [ ]:
train_null = pd.DataFrame(train.isna().sum())
train_null = train_null.sort_values(by = 0 ,ascending = False)[:-1] # [:-1]はtransported IDを抜かすため
train_null

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values")

In [ ]:
fig.add_trace(go.Bar(x=train_null[0],
                     y=train_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)' , 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 1)

In [ ]:
fig.add_trace(go.Bar(x=test_null[0], 
                     y=test_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)', 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 2)

In [ ]:
fig.update_layout(showlegend=False, title_text="Column wise Null Value Distribution", title_x=0.5)

### 4.3.2 Row wise Null Value Distribtion

In [ ]:
missing_train_row = train.isna().sum(axis=1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()
missing_test_row = test.isna().sum(axis=1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()
missing_train_row.columns = ['no', 'count']
missing_test_row.columns = ['no', 'count']
missing_train_row["count"] = missing_train_row["count"]*100
missing_test_row["count"] = missing_test_row["count"]*100


fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values",)

fig.add_trace(go.Bar(x=missing_train_row["no"], 
                     y=missing_train_row["count"]  ,
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)' ,
                                line_width = 3
                                ,coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x= missing_test_row["no"], 
                     y=missing_test_row["count"],
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)',
                                line_width = 3 ,
                                coloraxis="coloraxis")),
              1, 2)
fig.update_layout(showlegend=False, title_text="Row wise Null Value Distribution", title_x=0.5)

In [ ]:
missing_train_row = train.isna().sum(axis=1) # レコードごとの、missing dataの数を表示
missing_train_row

In [ ]:
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()

In [ ]:
missing_train_row

# 5. apendex Detecting missing data visually using Misiingno library

>The bar chart above gives a quick graphical overview of the completeness of the dataset. We can see that Age, Cabin and embarked columns have missing values. Next,it would make sense to find out the locations of the missing data.

In [ ]:
msno.bar(train)

## visualizing the locations of the missing data

>The [msno.matrix](https://github.com/ResidentMario/missingno#matrix) nullity matrix is a data-dense display which lets you quickly visually pick out patterns in data completion.

In [ ]:
msno.matrix(train)

>* The plot appears blank(white) wherever there are missing values. For instance, in HomePlanet column there are only ten instances of missing data, hence the ten white lines.
>
>* The sparkline on the right gives an idea of the general shape of the completeness of the data and points out the row with the minimum nullities and the total number of columns in a given dataset, at the bottom.
>
>It is also possible to sample the dataset to pinpoint the exact location of the missing values. For instance let's check the first 100 rows.

In [ ]:
msno.matrix(train.sample(100))

# Reasons for Missing Values 

>Before we start treating the missing values ,it is important to understand the various reasons for the missingness in data. Broadly speaking, there can be three possible reasons:

![](https://cjasn.asnjournals.org/content/clinjasn/early/2014/02/06/CJN.10141013/F2.large.jpg?width=800&height=600&carousel=1)

source:https://cjasn.asnjournals.org/content/early/2014/02/06/CJN.10141013/tab-figures-data?versioned=true


**1. Missing Completely at Random (MCAR) **

>The missing values on a given variable (Y) are not associated with other variables in a given data set or with the variable (Y) itself. In other words, there is no particular reason for the missing values.

**2. Missing at Random (MAR) **

>MAR occurs when the missingness is not random, but where missingness can be fully accounted for by variables where there is complete information.

**3. Missing Not at Random (MNAR) **
>Missingness depends on unobserved data or the value of the missing data itself. 

*All definitions taken from Wikipedia: https://en.wikipedia.org/wiki/Missing_data*


>Now let us look at nullity matrix again to see if can find what type of missingness is present in the dataset.


## Finding the reason for missing data using matrix plot 

In [ ]:
msno.matrix(train)

In [ ]:
#sorted by Age
sorted = train.sort_values('Age')
msno.matrix(sorted)

## Finding reason for missing data using a Heatmap

In [ ]:
msno.heatmap(train)

The heatmap function shows that there are no strong correlations between missing values of different features. This is good; low correlations further indicate that the data are MAR.

## Finding reason for misiing data using Dendrogram

In [ ]:
train_char = train.drop(labels = "Transported", axis= 1, inplace = False)
msno.dendrogram(train_char)